Hierbij wordt gebruik gemaakt van het package youtube-dl

In [ ]:
import numpy as np
import pandas as pd
import re
import csv
import datetime
import os
import wave
import subprocess
import glob
import youtube_dl

In [ ]:
## Lees alle csv-files in
bal_labels = pd.read_csv('csv_files/balanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

unbal_labels = pd.read_csv('csv_files/unbalanced_train_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

eval_labels = pd.read_csv('csv_files/eval_segments.csv', skiprows=3, 
                         quotechar='"', skipinitialspace = True, header=None, 
                         names = ["YTID", "start_seconds", "end_seconds", "positive_labels"])

In [ ]:
## Hierin worden de wav-files gegooid
file_path = "wav_files"

In [ ]:
## Maak folder aan als deze nog niet bestaat
if not os.path.isdir(file_path):
    os.makedirs(file_path)
    
## Maak ook folders aan voor bal/unbal/eval (om later makkelijk labels weer op te halen)
if not os.path.isdir(file_path + "/bal"):
    os.makedirs(file_path + "/bal")
if not os.path.isdir(file_path + "/unbal"):
    os.makedirs(file_path + "/unbal")
if not os.path.isdir(file_path + "/eval"):
    os.makedirs(file_path + "/eval")

In [ ]:
## Geef aan of de tf-records voor jungle of urban moeten worden gedownload (ik neem aan dat we alleen jungle doen)
target = "jungle" #"urban"
mid_to_label = pd.read_csv("csv_files/class_labels_indices_" + target + ".csv", sep=";")

In [ ]:
mid_to_label

In [ ]:
def getLabels(mid_str):
    ## Maak lijst van m-id's
    mid_list = mid_str.split(',')
    labels = []
    
    ## Voor elk m-id, vind labels, (if any labels: add to label list)
    for mid in mid_list:
        if (mid_to_label.loc[mid_to_label["mid"] == mid, "mid"].any()):
            labels.append(mid_to_label.loc[mid_to_label["mid"] == mid, "index"].values[0])
    
    ## Return unique set of labels
    return set(labels)

In [ ]:
## Download wav-files van youtube

def downloadWav(vid_to_mid, folder):

    for i in range(vid_to_mid.shape[0]):
        mids = vid_to_mid.loc[i, "positive_labels"]
        labels = getLabels(mids)

        if(len(labels)>0):

            url = vid_to_mid.loc[i, "YTID"]
            file_name = 'VID_' + url
            file_path = 'wav_files/' + folder
            
            ydl_opts = {
                'extractaudio': True,
                'format': 'bestaudio',
                'noplaylist': True,
                'outtmpl': file_path + file_name + '.%(ext)s',
#                 'quiet': True
            }
            
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download(['https://www.youtube.com/watch?v=' + url])
                                 
            # retrieve start of audio segment
            start_sec = vid_to_mid.loc[i, "start_seconds"]
           
            files = glob.glob(file_path + file_name + '.*')
            print(file_path + file_name + '.*')
            for f in files:
                print(f)
                file = f.replace('\\','/')
                path = file.rsplit('.',1)[0]

                subprocess.call('ffmpeg -ss '+ str(int(start_sec)) + ' -t 10 -i "'+ file +'" -vn -acodec pcm_u8 -ar 16000 "'+ path +'.wav"', shell=True);
            
                # TODO remove source file
#                 os.remove(file)

In [ ]:
## Hier wordt de functie aangeroepen en gaat het downloaden beginnen
downloadWav(bal_labels, "bal/")
#downloadWav(unbal_labels, "unbal/")
#downloadWav(eval_labels, "eval/")

In [ ]:
# TEST check file (length)
#f = wave.open(file_path + '.wav', 'r')
#frames = f.getnframes()
#rate = f.getframerate()
#duration = frames / float(rate)
#f.close()

#if(duration<10):
#    print("Warning - duration of video '" + url + "' is only " + str(duration) + " second(s)!")